# Catboost pred

Score: 167.50309

In [107]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [108]:
sys.path.append('../')
%load_ext autoreload
%autoreload 2
%store -r dm
dm = dm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Grabbing data, formatting
All features, 15min resolution, makima interpolated targets

In [109]:
makima_a = pd.read_csv("../csv_files/makima_a.csv")[['date_forecast', 'pv_measurement_interpolated']]
makima_b = pd.read_csv("../csv_files/makima_b.csv")[['date_forecast', 'pv_measurement_interpolated']]
makima_c = pd.read_csv("../csv_files/makima_c.csv")[['date_forecast', 'pv_measurement_interpolated']]

makima_a['date_forecast'] = makima_a['date_forecast'].astype('str')
makima_b['date_forecast'] = makima_b['date_forecast'].astype('str')
makima_c['date_forecast'] = makima_c['date_forecast'].astype('str')
makima_a

,date_forecast,pv_measurement_interpolated
0,2019-06-02 22:00:00,0.0
1,2019-06-02 22:15:00,0.0
2,2019-06-02 22:30:00,0.0
3,2019-06-02 22:45:00,0.0
4,2019-06-02 23:00:00,0.0
...,...,...
136329,2023-04-30 22:00:00,0.0
136330,2023-04-30 22:15:00,0.0
136331,2023-04-30 22:30:00,0.0
136332,2023-04-30 22:45:00,0.0


In [110]:
# This is ass. I want 15m resolution, but cant find it in data_A

X_A_obs = dm.X_train_observed_a
X_B_obs = dm.X_train_observed_b
X_C_obs = dm.X_train_observed_c

X_A_obs['est'] = 0
X_B_obs['est'] = 0
X_C_obs['est'] = 0

X_A_est = dm.X_train_estimated_a
X_B_est = dm.X_train_estimated_b
X_C_est = dm.X_train_estimated_c

X_A_est['est'] = 1
X_B_est['est'] = 1
X_C_est['est'] = 1

X_A = pd.concat([X_A_obs, X_A_est])
X_B = pd.concat([X_B_obs, X_B_est])
X_C = pd.concat([X_C_obs, X_C_est])

X_A['date_forecast'] = X_A['date_forecast'].astype('str')
X_B['date_forecast'] = X_B['date_forecast'].astype('str')
X_C['date_forecast'] = X_C['date_forecast'].astype('str')
X_A

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,est,date_calc
0,2019-06-02 22:00:00,7.7,1.230,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,...,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,0,NaT
1,2019-06-02 22:15:00,7.7,1.229,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,...,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,0,NaT
2,2019-06-02 22:30:00,7.7,1.228,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,...,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,0,NaT
3,2019-06-02 22:45:00,7.7,1.226,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,...,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,0,NaT
4,2019-06-02 23:00:00,7.7,1.225,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,...,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,2023-04-30 22:45:00,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,...,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0,1,2023-04-29 07:00:05
17572,2023-04-30 23:00:00,4.5,1.281,0.0,0.0,0.0,272.200012,0.0,0.0,0.0,...,0.1,274.200012,100.000000,9923.200195,3.7,2.8,2.4,-0.0,1,2023-04-29 07:00:05
17573,2023-04-30 23:15:00,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,...,0.1,274.200012,99.900002,11230.799805,3.7,2.7,2.5,-0.0,1,2023-04-29 07:00:05
17574,2023-04-30 23:30:00,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,...,0.1,274.200012,99.599998,12526.099609,3.8,2.7,2.6,-0.0,1,2023-04-29 07:00:05


In [111]:
# Merges X and targets
common_A = pd.merge(X_A, makima_a, how='inner', on='date_forecast')
common_B = pd.merge(X_B, makima_b, how='inner', on='date_forecast')
common_C = pd.merge(X_C, makima_c, how='inner', on='date_forecast')

X_A = X_A[X_A['date_forecast'].isin(common_A['date_forecast'])]
X_B = X_B[X_B['date_forecast'].isin(common_B['date_forecast'])]
X_C = X_C[X_C['date_forecast'].isin(common_C['date_forecast'])]

X_A = X_A.drop(['date_forecast', 'date_calc'], axis='columns')
X_B = X_B.drop(['date_forecast', 'date_calc'], axis='columns')
X_C = X_C.drop(['date_forecast', 'date_calc'], axis='columns')

X_A

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,est
0,7.7,1.230,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,...,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,0
1,7.7,1.229,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,...,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,0
2,7.7,1.228,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,...,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,0
3,7.7,1.226,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,...,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,0
4,7.7,1.225,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,...,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17568,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,0.0,...,-10.630,0.1,274.399994,97.400002,16674.900391,4.3,3.4,2.6,-0.0,1
17569,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,0.0,...,-11.102,0.1,274.399994,98.699997,15005.000000,4.2,3.3,2.6,-0.0,1
17570,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,0.0,...,-11.470,0.1,274.299988,99.500000,13323.200195,4.0,3.1,2.5,-0.0,1
17571,4.5,1.281,0.0,0.0,0.0,272.299988,0.0,0.0,0.0,0.0,...,-11.731,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0,1


In [112]:
y_A = makima_a[makima_a['date_forecast'].isin(common_A['date_forecast'])]['pv_measurement_interpolated']
y_B = makima_b[makima_b['date_forecast'].isin(common_B['date_forecast'])]['pv_measurement_interpolated']
y_C = makima_c[makima_c['date_forecast'].isin(common_C['date_forecast'])]['pv_measurement_interpolated']
y_A

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
136329    0.0
136330    0.0
136331    0.0
136332    0.0
136333    0.0
Name: pv_measurement_interpolated, Length: 136238, dtype: float64

## Catboost model

In [113]:
from sklearn.model_selection import train_test_split

X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X_A, y_A, shuffle=False, train_size=0.9)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(X_B, y_B, shuffle=False, train_size=0.9)
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C, y_C, shuffle=False, train_size=0.9)

print(X_train_A.shape, y_train_A.shape, X_test_A.shape, y_test_A.shape)
print(X_train_B.shape, y_train_B.shape, X_test_B.shape, y_test_B.shape)
print(X_train_C.shape, y_train_C.shape, X_test_C.shape, y_test_C.shape)

(122614, 43) (122614,) (13624, 43) (13624,)
(104741, 43) (104741,) (11638, 43) (11638,)
(83440, 43) (83440,) (9272, 43) (9272,)


### Time series

In [114]:
from catboost import CatBoostRegressor

catModel_A = CatBoostRegressor()
catModel_B = CatBoostRegressor()
catModel_C = CatBoostRegressor()

In [115]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=15,
    max_train_size=7000,
    test_size=1000,
)

all_splits_A = list(ts_cv.split(X_train_A, y_train_A))
all_splits_B = list(ts_cv.split(X_train_B, y_train_B))
all_splits_C = list(ts_cv.split(X_train_C, y_train_C))

In [116]:

from sklearn.metrics import mean_absolute_error
import pandas as pd

for train, test in (all_splits_A): 

    catModel_A.fit(X_train_A.iloc[train], y_train_A.iloc[train], silent=True, use_best_model=True, eval_set=(X_train_A.iloc[test], y_train_A.iloc[test]))

    preds_A = pd.DataFrame(catModel_A.predict(X_test_A))
    actual = y_test_A

    # preds_A = dm.scaling(preds_A, "A")
    # actual = dm.scaling(actual, "A") 

    print("CURRENT SCORING A ----->   ", mean_absolute_error(preds_A, actual))

for train, test in (all_splits_B): 

    catModel_B.fit(X_train_B.iloc[train], y_train_B.iloc[train], silent=True, use_best_model=True, eval_set=(X_train_B.iloc[test], y_train_B.iloc[test]))

    preds_B = pd.DataFrame(catModel_B.predict(X_test_B))
    actual = y_test_B

    # preds_B = dm.scaling(preds_B, "B")
    # actual = dm.scaling(actual, "B") 

    print("CURRENT SCORING B ----->   ", mean_absolute_error(preds_B, actual))

for train, test in (all_splits_C): 

    catModel_C.fit(X_train_C.iloc[train], y_train_C.iloc[train], silent=True, use_best_model=True, eval_set=(X_train_C.iloc[test], y_train_C.iloc[test]))

    preds_C = pd.DataFrame(catModel_C.predict(X_test_C))
    actual = y_test_C

    # preds_C = dm.scaling(preds_C, "C")
    # actual = dm.scaling(actual, "C") 

    print("CURRENT SCORING C ----->   ", mean_absolute_error(preds_C, actual))



CURRENT SCORING A ----->    180.81426573202427
CURRENT SCORING A ----->    210.0577058189216
CURRENT SCORING A ----->    219.79661513654972
CURRENT SCORING A ----->    211.42919129845134
CURRENT SCORING A ----->    205.83629833456155
CURRENT SCORING B ----->    41.79018894131748
CURRENT SCORING B ----->    33.31735775736036
CURRENT SCORING B ----->    32.102544423072494
CURRENT SCORING B ----->    30.410017373946864
CURRENT SCORING B ----->    30.446372393890133
CURRENT SCORING C ----->    42.01008470838956
CURRENT SCORING C ----->    28.253323989458416
CURRENT SCORING C ----->    27.30614796984384
CURRENT SCORING C ----->    27.6438889825535
CURRENT SCORING C ----->    27.079622994392285


### Normal splits

In [117]:
catModel_A.fit(X_train_A, y_train_A, silent=True)
catModel_B.fit(X_train_B, y_train_B, silent=True)
catModel_C.fit(X_train_C, y_train_C, silent=True)

In [118]:
import pandas as pd 
from sklearn.metrics import mean_absolute_error
preds_A = pd.DataFrame(catModel_A.predict(X_test_A))
actual = y_test_A

# preds_A = dm.scaling(preds_A, "A")
# actual = dm.scaling(actual, "A") 

print("CURRENT SCORING A ----->   ", mean_absolute_error(actual, preds_A))

preds_B = pd.DataFrame(catModel_B.predict(X_test_B))
actual = y_test_B

# preds_B = dm.scaling(preds_B, "B")
# actual = dm.scaling(actual, "B") 

print("CURRENT SCORING B ----->   ", mean_absolute_error(actual, preds_B))

preds_C = pd.DataFrame(catModel_C.predict(X_test_C))
actual = y_test_C

# preds_C = dm.scaling(preds_C, "C")
# actual = dm.scaling(actual, "C") 

print("CURRENT SCORING C ----->   ", mean_absolute_error(actual, preds_C))

CURRENT SCORING A ----->    146.97237601257362
CURRENT SCORING B ----->    21.694197550645256
CURRENT SCORING C ----->    24.839228537058144


## Create submission

In [125]:
catModel_A.fit(X_A, y_A, silent=True)
catModel_B.fit(X_B, y_B, silent=True)
catModel_C.fit(X_C, y_C, silent=True)

In [144]:
X_test_estimated_a = pd.read_parquet('../A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../C/X_test_estimated.parquet')

X_A_submission = X_test_estimated_a[X_test_estimated_a.columns.intersection(X_A.columns)]
X_B_submission = X_test_estimated_b[X_test_estimated_b.columns.intersection(X_B.columns)]
X_C_submission = X_test_estimated_c[X_test_estimated_c.columns.intersection(X_C.columns)]

X_A_submission['est'] = 1
X_B_submission['est'] = 1
X_C_submission['est'] = 1

/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_81854/2095999153.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_A_submission['est'] = 1
/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_81854/2095999153.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_B_submission['est'] = 1
/var/folders/kq/hk1l39ys077bc7f9741ypg800000gn/T/ipykernel_81854/2095999153.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [147]:
X_test_estimated_a

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,...,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,...,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,...,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,...,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,...,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-02 07:00:31,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,...,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-02 07:00:31,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,...,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-02 07:00:31,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,...,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-02 07:00:31,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,...,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


In [145]:
## Specify the model to be used
pred_A = catModel_A.predict(X_A_submission)
pred_B = catModel_B.predict(X_B_submission)
pred_C = catModel_C.predict(X_C_submission)

## Uncomment if normalizing data
# pred_A = dm.scaling(pred_A, "A")
# pred_B = dm.scaling(pred_B, "B")
# pred_C = dm.scaling(pred_C, "C")

test = pd.read_csv('../test.csv')

df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)


# df["id"] = test["id"]

# df = df[["id", "prediction"]]

df[df<0] = 0


In [150]:
# Down-sample submission
def average_every_four_rows(df):
    # Group every 4 rows and take the mean of the 'prediction' column
    averaged_values = df['prediction'].groupby(df.index // 4).mean().reset_index(drop=True)
    
    # Create a new dataframe with the averaged values
    averaged_df = pd.DataFrame({'prediction': averaged_values})
    
    return averaged_df

# Compute the average for every 4 rows
averaged_df = average_every_four_rows(df)

# Check the shape of the resulting dataframe
averaged_df.shape, averaged_df.head()

((2160, 1),
    prediction
 0    1.572870
 1    2.633185
 2    9.198144
 3  128.018209
 4  529.207872)

In [151]:
# NAME THE FILE 
averaged_df.to_csv("teo_subs/teo_sub_1.csv")